In [ ]:
!pip install ibm_db==2.0.8a
!pip install flask-ngrok
!pip install flask-bootstrap
!pip install pyngrok==4.1.1
!ngrok authtoken '1tShcLWOqT6sVMKWl6qo37PDosF_5kP7mo4kKjocvFq6k8fWp'
!pip install google-search-results

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 689 kB 25.8 MB/s 
  Created wheel for ibm-db: filename=ibm_db-2.0.8a0-cp37-cp37m-linux_x86_64.whl size=41343728 sha256=58f516f96d3debb249d229d42a2580ba1874f098682e8f403dbb3a001f67b0ab
  Stored in directory: /root/.cache/pip/wheels/b7/30/6e/c3a4ed63e5a80f90d3456943e4d5c5f31efc5a5c4249038d6f
Successfully built ibm-db
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 456 kB 25.8 MB/s 
  Created wheel for flask-bootstrap: filename=Flask_Bootstrap-3.3.7.1-py3-none-any.whl size=460123 sha256=93ca299701e91add42ac1bc2e5ef99a1059d479acbb5eb7c33b55f37f9b676e9
  Stored in directory: /root/.cache/pip/wheels/67/a2/d6/50d039c9b59b4caca6d7b53839c8100354a52ab7553d2456eb
  Created 

In [ ]:
import ibm_db
conn = ibm_db.connect("DATABASE=bludb;HOSTNAME=9938aec0-8105-433e-8bf9-0fbb7e483086.c1ogj3sd0tgtu0lqde00.databases.appdomain.cloud;PORT=32459;SECURITY=SSL;PROTOCOL=TCPIP;UID=tmq90769;PWD=Iff2xEpAjAPd0hD0",'','')

In [16]:
from flask import Flask,render_template,jsonify,json,request
from flask_ngrok import run_with_ngrok
from googlesearch import search
from serpapi import GoogleSearch
app = Flask(__name__)
run_with_ngrok(app)

@app.route("/")
def home():
	return render_template('chatbot.html')
 
@app.route("/adduser")
def hom():
	return render_template('adduser.html') 
@app.route("/add_user",methods = ['POST','GET'])
def adduser():
  name=request.form['name']
  age=request.form['age']
  skills=request.form['skills']
  q=request.form['q']
  query = "INSERT into users values(?,?,?,?)"
  stmt = ibm_db.prepare(conn, query)
  ibm_db.bind_param(stmt, 1, name)
  ibm_db.bind_param(stmt, 2, age)
  ibm_db.bind_param(stmt, 3, skills)
  ibm_db.bind_param(stmt, 4, q)
  ibm_db.execute(stmt)
  return "Success"
@app.route("/candidate_list")
def blue():
  id=[]
  name=[]
  skill=[]
  experience=[]
  gender=[]
  email=[]
  phno=[]
  address=[]
  salary=[]

  query = "SELECT * FROM candidate"
  stmt = ibm_db.prepare(conn, query)
  ibm_db.execute(stmt)
  dictionary = ibm_db.fetch_assoc(stmt)
  while dictionary!= False:
    name.append(dictionary['NAME'])
    skill.append(dictionary['SKILLS'])
    id.append(dictionary['ID'])
    experience.append(dictionary['EXPERIENCE'])
    gender.append(dictionary['GENDER'])
    email.append(dictionary['EMAIL'])
    phno.append(dictionary['PHNO'])
    salary.append(dictionary['SALARY'])
    address.append(dictionary['ADDRESS'])

    dictionary = ibm_db.fetch_assoc(stmt)
  return render_template('candidate.html',names=name,skills=skill,id=id,gender=gender,
                         email=email,address=address,salary=salary,experience=experience,phno=phno)  
@app.route("/user_list")
def red():
  name=[]
  skills=[]

  query = "SELECT * FROM users"
  stmt = ibm_db.prepare(conn, query)
  ibm_db.execute(stmt)
  dictionary = ibm_db.fetch_assoc(stmt)
  while dictionary!= False:
    name.append(dictionary['NAME'])
    skills.append(dictionary['SKILLS'])
    dictionary = ibm_db.fetch_assoc(stmt)
  return render_template('job_list.html',names=name,skills=skills) 
@app.route("/login")
def login():
  return render_template('login.html') 
@app.route("/signup")
def reg():
  return render_template('registration.html')   
@app.route("/apply")
def html():
  return render_template('applyjobs.html')  
@app.route("/search")
def search():
  return render_template('jobsearch.html')  
@app.route("/assistant")
def assi():
  return render_template('chatbot.html')  
@app.route("/apply_jobs",methods = ['POST','GET'])
def server():
    id = request.form['id']
    name = request.form['name']
    skills = request.form['skills']
    exp = request.form['exp']
    gender = request.form['gender']
    mail = request.form['email']
    phno = request.form['phno']
    address = request.form['address']
    salary = request.form['salary']
    query = "INSERT into candidate values(?,?,?,?,?,?,?,?,?)"
    stmt = ibm_db.prepare(conn, query)
    ibm_db.bind_param(stmt, 1, id)
    ibm_db.bind_param(stmt, 2, name)
    ibm_db.bind_param(stmt, 3, skills)
    ibm_db.bind_param(stmt, 4, exp)
    ibm_db.bind_param(stmt, 5, gender)
    ibm_db.bind_param(stmt, 6, mail)
    ibm_db.bind_param(stmt, 7, phno)
    ibm_db.bind_param(stmt, 8, address)
    ibm_db.bind_param(stmt, 9, salary)
    ibm_db.execute(stmt)
    return "success";    

@app.route("/api",methods = ['POST','GET'])
def api():
  a=request.form['search']
  search = GoogleSearch({  "engine": "google_jobs","q": a,  "api_key": "2d444c5f42ee2e27cad08df5c61a8d706cecd33179d1db4fd1dbfe5a358c541a"})
  result = search.get_json()
  r=json.dumps(result)
  r=json.loads(r)
  return render_template('api.html',title1=r["jobs_results"][0]["title"],company1=r["jobs_results"][0]["company_name"],
                        description1=r["jobs_results"][0]["description"],location1=r["jobs_results"][0]["location"],
                        title2=r["jobs_results"][0]["title"],company2=r["jobs_results"][0]["company_name"],
                        description2=r["jobs_results"][0]["description"],location2=r["jobs_results"][0]["location"],
                         title3=r["jobs_results"][0]["title"],company3=r["jobs_results"][0]["company_name"],
                        description3=r["jobs_results"][0]["description"],location3=r["jobs_results"][0]["location"],
                         title4=r["jobs_results"][0]["title"],company4=r["jobs_results"][0]["company_name"],
                        description4=r["jobs_results"][0]["description"],location4=r["jobs_results"][0]["location"]
                         ,
                         title5=r["jobs_results"][0]["title"],company5=r["jobs_results"][0]["company_name"],
                        description5=r["jobs_results"][0]["description"],location5=r["jobs_results"][0]["location"]); 

app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://6ef0-34-86-193-171.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:45:35] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:45:36] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:45:46] "GET /signup HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:45:46] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:46:11] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:46:30] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:47:13] "GET /adduser HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:47:29] "GET /user_list HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:47:50] "GET /apply HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:48:23] "POST /apply_jobs HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:48:34] "GET /candidate_list HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:49:04] "GET /search HTTP/1.1" 200 -


https://serpapi.com/search


INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:49:13] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:49:28] "POST /api HTTP/1.1" 200 -


https://serpapi.com/search
https://serpapi.com/search


INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:49:40] "POST /api HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [19/Nov/2022 05:49:50] "GET / HTTP/1.1" 200 -
